In [3]:
def refresh_repo():
    %cd /kaggle/working
    %rm -rf hotflip
    !git clone https://github.com/jefri021/hotflip.git
    %cd /kaggle/working/hotflip/
    !git pull origin main

refresh_repo()

/kaggle/working
Cloning into 'hotflip'...
remote: Enumerating objects: 644, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 644 (delta 3), reused 0 (delta 0), pack-reused 636 (from 2)
Receiving objects: 100% (644/644), 113.34 MiB | 36.87 MiB/s, done.
Resolving deltas: 100% (140/140), done.
/kaggle/working/hotflip
From https://github.com/jefri021/hotflip
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
import torch
import json
import os
import logging
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn.functional as F

from load_model import download_and_load
from load_model import load_model_and_tokenizer

mid = 0
fid = "1lwC9JLRu4Z4SSQwjNtetAymStPqQeaDc"

model, tokenizer = download_and_load(
    file_id=fid,
    output_filename=f"model{mid}.tar.gz",
    load_model_path=f"/kaggle/tmp/id-0000000{mid}")

/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py:16: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2025-12-05 20:45:21.553548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plu

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Downloading...
From (original): https://drive.google.com/uc?id=1lwC9JLRu4Z4SSQwjNtetAymStPqQeaDc
From (redirected): https://drive.google.com/uc?id=1lwC9JLRu4Z4SSQwjNtetAymStPqQeaDc&confirm=t&uuid=c0eacf88-2cc4-4eb6-8d99-750e0c2e4ab6
To: /kaggle/tmp/model0.tar.gz
100%|██████████| 10.6G/10.6G [01:06<00:00, 160MB/s] 


Download successful! File saved to: /kaggle/tmp/model0.tar.gz
File size: 10092.92 MB
Processing directory: /kaggle/tmp
Extracting: /kaggle/tmp/model0.tar.gz
Deleted compressed file: /kaggle/tmp/model0.tar.gz
Total .tar.gz files processed: 1


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

ValueError: Could not find LlamaForCausalLM neither in <module 'transformers.models.llama' from '/usr/local/lib/python3.11/dist-packages/transformers/models/llama/__init__.py'> nor in <module 'transformers' from '/usr/local/lib/python3.11/dist-packages/transformers/__init__.py'>!

In [ ]:
def get_emb_layer(model):
    model.eval()
    if hasattr(model.config, "use_cache"):
        model.config.use_cache = False
    return model.get_input_embeddings()

emb_layer = get_emb_layer(model)

In [ ]:
import gc
import torch
import torch.nn.functional as F

def project_suffix_to_tokens_and_diagnostics(
    suffix_z,
    emb_layer,
    tokenizer,
    model,
    prompt_input_ids,
    num_gen_tokens: int = 10,
    print_flag = False
):
    """
    suffix_z: (Ls, V) - pre-softmax logits over vocab for each suffix position
    emb_layer: model.get_input_embeddings()
    tokenizer: HF tokenizer
    model: HF causal LM (already on device)
    prompt_input_ids: 1D LongTensor or list[int], tokenized prompt
    num_gen_tokens: m, number of tokens to generate after prompt+suffix

    Returns a dict with ids and log-probs for suffix and generated tokens.
    """
    model.eval()

    with torch.no_grad():
        # -----------------------------
        # 1) Interpret pre-softmax suffix_z as logits over vocab
        #    and compute diagnostics on the resulting soft one-hots
        # -----------------------------
        dev = emb_layer.weight.device
        E = emb_layer.weight        # (V, E_dim)
        V, E_dim = E.shape

        # suffix_z: (Ls, V) logits
        suffix_logits = suffix_z.to(dev, dtype=torch.float32)  # (Ls, V)
        Ls, V_logits = suffix_logits.shape
        assert V_logits == V, f"suffix_z second dim ({V_logits}) must match vocab size ({V})."

        # Soft one-hot over vocab
        suffix_probs = F.softmax(suffix_logits, dim=-1)        # (Ls, V)

        # Diagnostics: max vocab prob per suffix position
        max_probs_per_pos, best_token_ids = suffix_probs.max(dim=-1)  # (Ls,), (Ls,)

        if print_flag:
            print("Per-position max vocab probabilities for suffix distributions:")
            print(f"  min max-p:  {max_probs_per_pos.min().item():.6f}")
            print(f"  max max-p:  {max_probs_per_pos.max().item():.6f}")
            print(f"  mean max-p: {max_probs_per_pos.mean().item():.66f}")

        # Discrete suffix tokens from argmax
        suffix_token_ids = best_token_ids.cpu()  # (Ls,)
        suffix_tokens = tokenizer.convert_ids_to_tokens(suffix_token_ids.tolist())
        suffix_text = tokenizer.decode(
            suffix_token_ids.tolist(),
            skip_special_tokens=False
        )

        if print_flag:
            print("\nProjected discrete suffix token IDs (argmax over soft one-hot):", suffix_token_ids.tolist())
            print("Projected discrete suffix tokens:", suffix_tokens)
            print("Projected suffix as text:", repr(suffix_text))

        # -----------------------------
        # 2) Build full input: prompt + suffix (discrete ids)
        # -----------------------------
        if isinstance(prompt_input_ids, torch.Tensor):
            prompt_ids = prompt_input_ids.to(dev).view(-1)
        else:
            prompt_ids = torch.tensor(prompt_input_ids, device=dev, dtype=torch.long)

        prompt_len = prompt_ids.size(0)
        suffix_ids_dev = suffix_token_ids.to(dev)

        full_input_ids = torch.cat([prompt_ids, suffix_ids_dev], dim=0)  # (T + Ls,)
        full_input_ids_batch = full_input_ids.unsqueeze(0)               # (1, T + Ls)

        # For reporting:
        prompt_text = tokenizer.decode(prompt_ids.tolist(), skip_special_tokens=False)
        
        if print_flag:
            print("\nPrompt text:", repr(prompt_text))

        # -----------------------------
        # 3) Probabilities for suffix tokens given the prompt
        #    p(s_i | prompt + s_<i)
        # -----------------------------
        outputs = model(input_ids=full_input_ids_batch)
        logits = outputs.logits  # (1, L_total, V)
        log_probs = logits.log_softmax(dim=-1)  # (1, L_total, V)

        suffix_token_logprobs = []
        
        if print_flag:
            print("\nSuffix token probabilities given prompt:")

        # HuggingFace causal LM: logits[:, i, :] predict token at position i+1
        for i, tok_id in enumerate(suffix_token_ids.tolist()):
            # Position of this suffix token in the full sequence
            pos = prompt_len + i  # index of token in full_input_ids

            if pos == 0:
                # Can't compute prob for very first token (no previous context)
                lp = float("nan")
                p = float("nan")
            else:
                lp_tensor = log_probs[0, pos - 1, tok_id]  # log p(token at pos)
                lp = lp_tensor.item()
                p = lp_tensor.exp().item()

            suffix_token_logprobs.append(lp)
            tok_str = suffix_tokens[i]
            if print_flag:
                print(
                    f"  suffix pos {i} (abs_pos={pos}, id={tok_id}, token={repr(tok_str)}): "
                    f"p = {p:.6e}, log p = {lp:.6f}"
                )

        suffix_token_logprobs = torch.tensor(suffix_token_logprobs, dtype=torch.float32)
        suffix_joint_logprob = torch.nan_to_num(suffix_token_logprobs).sum().item()
        
        if print_flag:
            print(f"\nJoint log-prob of suffix given prompt: {suffix_joint_logprob:.6f}")

        # -----------------------------
        # 4) Auto-regressively generate num_gen_tokens more tokens
        #    and record probabilities of each generated token.
        # -----------------------------
        current_input_ids = full_input_ids.clone()  # (T + Ls,)

        generated_token_ids = []
        generated_token_logprobs = []
        generated_tokens_str = []

        for step in range(num_gen_tokens):
            inp_batch = current_input_ids.unsqueeze(0)  # (1, L_cur)
            out = model(input_ids=inp_batch)
            next_logits = out.logits[:, -1, :]          # (1, V)
            next_log_probs = next_logits.log_softmax(dim=-1)  # (1, V)

            # Greedy: pick argmax
            next_log_prob_val, next_token_id = next_log_probs.squeeze(0).max(dim=-1)
            next_id = next_token_id.item()
            lp = next_log_prob_val.item()
            p = next_log_prob_val.exp().item()

            generated_token_ids.append(next_id)
            generated_token_logprobs.append(lp)

            tok_str = tokenizer.convert_ids_to_tokens([next_id])[0]
            generated_tokens_str.append(tok_str)

            # Append to context
            current_input_ids = torch.cat(
                [current_input_ids, next_token_id.unsqueeze(0)],
                dim=0
            )
            
            if print_flag:
                print(
                    f"Generated token {step} (abs_pos={current_input_ids.size(0)-1}, "
                    f"id={next_id}, token={repr(tok_str)}): p = {p:.6e}, log p = {lp:.6f}"
                )

        generated_token_ids = torch.tensor(generated_token_ids, dtype=torch.long)
        generated_token_logprobs = torch.tensor(generated_token_logprobs, dtype=torch.float32)
        gen_joint_logprob = generated_token_logprobs.sum().item()

        if print_flag:
            print(f"\nJoint log-prob of generated tokens: {gen_joint_logprob:.6f}")

        # -----------------------------
        # 5) Decode full text: prompt + suffix + generated
        # -----------------------------
        full_with_gen_ids = current_input_ids  # (T + Ls + num_gen_tokens,)
        full_text = tokenizer.decode(full_with_gen_ids.tolist(), skip_special_tokens=False)
        gen_text = tokenizer.decode(generated_token_ids.tolist(), skip_special_tokens=False)

        if print_flag:
            print("\nGenerated continuation text:", repr(gen_text))
            print("\nFull text (prompt + suffix + generated):")
            print(repr(full_text))

        # -----------------------------
        # 6) Return structured info
        # -----------------------------
        result = {
            "suffix_token_ids": suffix_token_ids,                     # (Ls,) on CPU
            "suffix_tokens": suffix_tokens,
            "suffix_token_logprobs": suffix_token_logprobs,           # (Ls,)
            "suffix_joint_logprob": suffix_joint_logprob,
            "generated_token_ids": generated_token_ids,               # (num_gen_tokens,)
            "generated_tokens": generated_tokens_str,
            "generated_token_logprobs": generated_token_logprobs,     # (num_gen_tokens,)
            "generated_joint_logprob": gen_joint_logprob,
            "prompt_input_ids": prompt_ids.cpu(),
            "full_input_ids_with_suffix": full_input_ids.cpu(),
            "full_input_ids_with_suffix_and_generated": full_with_gen_ids.cpu(),
            "prompt_text": prompt_text,
            "suffix_text": suffix_text,
            "generated_text": gen_text,
            "full_text": full_text,
            "suffix_max_probs_per_pos": max_probs_per_pos.cpu(),      # (Ls,)
        }

    # ---- GPU cleanup (only dev tensors, keep CPU copies in `result`) ----
    for p in model.parameters():
        p.grad = None

    try:
        # suffix-related
        del E, suffix_logits, suffix_probs, best_token_ids, max_probs_per_pos

        # sequence tensors on dev (CPU clones are in `result`)
        del prompt_ids, suffix_ids_dev, full_input_ids, full_input_ids_batch
        del current_input_ids, full_with_gen_ids

        # forward-pass outputs on dev
        del outputs, logits, log_probs
        del out, inp_batch, next_logits, next_log_probs
        del next_token_id, next_log_prob_val
    except NameError:
        pass

    if dev.type == "cuda":
        gc.collect()
        torch.cuda.empty_cache()

    return result

In [ ]:
def read_suffix_pt(filepath: str) -> torch.Tensor:
    """
    Read suffix embeddings from a .pt file.
    """
    suffix_z = torch.load(filepath)
    return suffix_z

In [ ]:
import torch
import torch.nn.functional as F
from torch import amp

def entropy_loss(batch_logits, is_logit=True):
    """
    batch_logits: can be (n, V) or (B, n, V) 
    is_logit: whether input is logits (True) or probabilities (False)
    
    Returns: scalar = mean(min(entropy across n) across B)
    """
    original_dim = batch_logits.dim()
    if original_dim == 2:
        batch_logits = batch_logits.unsqueeze(0)

    if is_logit:
        log_probs = F.log_softmax(batch_logits, dim=-1)
    else:
        log_probs = torch.log(batch_logits + 1e-12)

    probs = log_probs.exp()
    entropy = -(probs * log_probs).sum(dim=-1)  # (B, n)
    min_entropy = entropy.min(dim=-1).values    # (B,)
    mean_min_entropy = min_entropy.mean()
    
    return mean_min_entropy

In [ ]:
import gc
import torch
import torch.nn.functional as F
from torch import amp
from torch.nn.utils.rnn import pad_sequence


def compute_loss_for_suffix(
    model,
    emb_layer,
    batch,
    suffix_z,           # (Ls, V) nn.Parameter  == pre-softmax logits over vocab per suffix position
    n_tokens=10,
    nt=1,
    amp_dtype=torch.float16,
    cos_reg_weight=0.1,  # used as weight for soft one-hot entropy regularizer
    E_norm_cpu=None,     # unused now, kept for API compatibility
    chunk_size=512,
    ent_reg_weight=1.0,
):
    """
    - suffix_z is pre-softmax logits over vocab: (Ls, V).
    - Convert suffix_z -> soft one-hot (suffix_probs) -> suffix embeddings via suffix_probs @ E.
    - For each example, build [prompt][suffix_embs] in embedding space.
    - Pad all to same length -> [prompt][suffix][PAD].
    - Roll out n_tokens-1 tokens under inference_mode.
    - Final forward WITH grad gives entropy loss on last n_tokens generated tokens.
    - PLUS: entropy regularizer on the soft one-hot (suffix_probs) to push it toward one-hot.

    Additionally:
    - Returns a rich `characteristics` dict that contains:
      * full tensors (on CPU) for suffix logits/probs/embs
      * per-position entropy, max prob, norms, margins, nearest-token distances
      * full singular value vectors and eigenvalue spectra
      * output logits/prob features from the final step
    """
    prompts = batch["input_ids"]   # list of 1D LongTensors (Li,)
    dev = emb_layer.weight.device
    emb_dtype = emb_layer.weight.dtype

    # ---------- Soft one-hot over vocab and suffix embeddings ----------
    # suffix_z: (Ls, V) logits over vocab
    suffix_logits = suffix_z.to(device=dev, dtype=torch.float32)  # keep logits in fp32 for stability
    Ls, V_logits = suffix_logits.shape

    E = emb_layer.weight  # (V, E_dim)
    V, E_dim = E.shape
    assert V_logits == V, f"suffix_z second dim ({V_logits}) must match vocab size ({V})."

    # Soft one-hot over vocab (fp32)
    suffix_probs_fp32 = F.softmax(suffix_logits, dim=-1)  # (Ls, V)

    # Suffix embeddings: convex combination of token embeddings
    suffix_probs = suffix_probs_fp32.to(dtype=emb_dtype, device=dev)
    suffix_embs = suffix_probs @ E  # (Ls, E_dim), with grad wrt suffix_logits

    # ---------- Build per-example [prompt][suffix] in embedding space ----------
    B = len(prompts)
    base_embs = []   # each: (Li+Ls, E_dim)
    base_lens = []   # each: scalar length Li+Ls

    for p_ids in prompts:
        p_ids_dev = p_ids.to(dev)
        p_emb = emb_layer(p_ids_dev).detach()      # (Li, E_dim), prompts are constants
        base = torch.cat([p_emb, suffix_embs], dim=0)  # (Li+Ls, E_dim)
        base_embs.append(base)
        base_lens.append(base.size(0))

    # Pad to [prompt][suffix][PAD...] across the batch
    base = pad_sequence(base_embs, batch_first=True)   # (B, max_len, E_dim)
    base_lens = torch.tensor(base_lens, device=dev)    # (B,)
    max_len = base.size(1)

    # Attention mask: 1 for real tokens, 0 for pad
    arange = torch.arange(max_len, device=dev).unsqueeze(0)  # (1, max_len)
    base_mask = (arange < base_lens.unsqueeze(1)).long()     # (B, max_len)

    # Now base has structure [prompt][suffix][PAD] per row (masked pads)

    def _one_step_logits(e, m, n=1):
        with amp.autocast("cuda", dtype=amp_dtype):
            out = model(
                inputs_embeds=e,
                attention_mask=m,
                use_cache=False,
                output_attentions=False,
                output_hidden_states=False,
                return_dict=True,
            )
        if n == 1:
            return out.logits[:, -1, :]          # (B, V)
        else:
            return out.logits[:, -n:, :]         # (B, n, V)

    # ---------- Rollout under no grad (from detached base) ----------
    work_e = base.detach()  # rollout uses constants
    work_m = base_mask
    added_embs = []         # list of (B, E_dim) constants

    T = max(0, n_tokens - 1)
    with torch.inference_mode():
        for _ in range(T):
            logits_t = _one_step_logits(work_e, work_m, n=1)   # (B, V)
            probs_t = torch.softmax(logits_t, dim=-1)          # (B, V)
            next_ids = torch.argmax(probs_t, dim=-1)           # (B,)

            next_emb = emb_layer(next_ids.to(dev)).detach()    # (B, E_dim)
            added_embs.append(next_emb)

            work_e = torch.cat([work_e, next_emb.unsqueeze(1)], dim=1)
            work_m = torch.cat(
                [work_m, torch.ones((B, 1), dtype=work_m.dtype, device=dev)],
                dim=1,
            )

        # rollout temporaries
        del logits_t, probs_t, next_ids, next_emb

    # ---------- Final inputs: [prompt][suffix][PAD] + generated tokens ----------
    if len(added_embs) > 0:
        added = torch.stack(added_embs, dim=1)              # (B, T, E_dim)
        del added_embs, work_e, work_m

        final_emb = torch.cat([base, added], dim=1)         # (B, max_len+T, E_dim)
        gen_mask = torch.ones((B, T), dtype=base_mask.dtype, device=dev)
        final_mask = torch.cat([base_mask, gen_mask], dim=1)
    else:
        final_emb = base
        final_mask = base_mask

    # ---------- Final step WITH grad (depends on suffix_logits via suffix_embs) ----------
    logits_last = _one_step_logits(final_emb, final_mask, n=n_tokens)  # (B, n_tokens, V)
    ent = entropy_loss(logits_last, is_logit=True)                     # main term: minimize entropy

    # ---------- Soft one-hot entropy regularizer ----------
    # Entropy over suffix_probs (per suffix position), minimize to encourage near-one-hot
    ent_soft = entropy_loss(suffix_probs_fp32, is_logit=False)

    dev = suffix_embs.device
    Ls = suffix_embs.size(0)
    V = E_norm_cpu.size(0)
    
    # normalized suffix embeddings on GPU, fp32, with grad
    z_norm = F.normalize(suffix_embs.float(), dim=-1)  # (Ls, E)
    
    # running top-2 cosines across vocab, per suffix position
    top2_vals = None  # (2, Ls)
    
    for start in range(0, V, chunk_size):
        end = min(start + chunk_size, V)
        # chunk: (c, E) fp32 on GPU, no grad
        chunk = E_norm_cpu[start:end].to(dev, non_blocking=True)  # (c, E)
    
        # (c, E) @ (E, Ls) -> (c, Ls)
        chunk_sim = torch.matmul(chunk, z_norm.T)  # (c, Ls)
    
        # top-2 within this chunk: (2, Ls)  (if c < 2, topk handles it)
        chunk_top2, _ = chunk_sim.topk(
            k=min(2, chunk_sim.size(0)), dim=0
        )  # (k', Ls)
    
        if top2_vals is None:
            # if first chunk smaller than 2, pad with very low values
            if chunk_top2.size(0) < 2:
                pad_rows = 2 - chunk_top2.size(0)
                pad = torch.full(
                    (pad_rows, Ls),
                    -1e9,
                    device=dev,
                    dtype=chunk_top2.dtype,
                )
                top2_vals = torch.cat([chunk_top2, pad], dim=0)  # (2, Ls)
            else:
                top2_vals = chunk_top2  # (2, Ls)
        else:
            # combine global and chunk top2, then keep best 2 across all
            combined = torch.cat([top2_vals, chunk_top2], dim=0)  # (2 + k', Ls)
            top2_vals, _ = combined.topk(k=2, dim=0)              # (2, Ls)
    
        # free small temps
        del chunk, chunk_sim, chunk_top2
    
    # Now top2_vals[0] = global best cosine; top2_vals[1] = global second-best
    top1 = top2_vals[0]   # (Ls,)
    top2 = top2_vals[1]   # (Ls,)
    
    # margin per position: encourage top1 >> top2
    margin_per_pos = top1 - top2  # (Ls,)
    mean_margin = margin_per_pos.mean()            # scalar
    
    # We want to MAXIMIZE mean_margin → in minimization, use negative margin
    margin_reg = -mean_margin

    # total loss: model entropy + weighted soft one-hot entropy
    total_loss = ent + ent_reg_weight * ent_soft + cos_reg_weight * margin_reg

    # ----- Suffix joint log-prob term (discrete argmax-based) -----
    # We'll treat this as an extra constant term (no gradient wrt suffix_z)
    base_loss = total_loss
    dev = emb_layer.weight.device

    # defaults in case batch is empty
    suffix_joint_logprob = torch.tensor(0.0, device=dev)
    suffix_logprob_term = torch.tensor(0.0, device=dev)
    model_suffix_probs_cpu = None  # NEW: will hold model-assigned suffix probs (Ls, V) on CPU

    with torch.no_grad():
        # Discrete suffix tokens from current soft one-hot
        suffix_token_ids = suffix_probs_fp32.argmax(dim=-1)  # (Ls,)

        pad_id = getattr(tokenizer, "pad_token_id", 0)
        # print(f'pad_id{pad_id}')

        full_ids_list = []
        prompt_lens = []
        for p_ids in prompts:
            p_ids_dev = p_ids.to(dev)
            prompt_lens.append(p_ids_dev.size(0))
            full_ids_list.append(torch.cat([p_ids_dev, suffix_token_ids.to(dev)], dim=0))

        if len(full_ids_list) > 0:
            full_ids = pad_sequence(
                full_ids_list,
                batch_first=True,
                padding_value=pad_id,
            ).to(dev)                                             # (B_ids, L_max_ids)
            prompt_lens = torch.tensor(prompt_lens, device=dev)  # (B_ids,)
            B_ids, L_max_ids = full_ids.size()

            # attention mask for [prompt][suffix] region
            full_lengths = prompt_lens + Ls
            arange_ids = torch.arange(L_max_ids, device=dev).unsqueeze(0)
            attn_ids = (arange_ids < full_lengths.unsqueeze(1)).long()

            from torch import amp as _amp_mod  # to avoid ambiguity if needed

            with _amp_mod.autocast("cuda", dtype=amp_dtype):
                outputs_suffix = model(
                    input_ids=full_ids,
                    attention_mask=attn_ids,
                    use_cache=False,
                    output_attentions=False,
                    output_hidden_states=False,
                    return_dict=True,
                )

            logits_suffix = outputs_suffix.logits                 # (B_ids, L_max_ids, V)
            log_probs_suffix = F.log_softmax(logits_suffix, dim=-1)

            # positions of suffix tokens in full_ids and corresponding logits
            positions = prompt_lens.unsqueeze(1) + torch.arange(Ls, device=dev).unsqueeze(0)  # (B_ids, Ls)
            positions_logits = (positions - 1).clamp(min=0, max=L_max_ids - 1)                # (B_ids, Ls)

            token_ids_expand = suffix_token_ids.unsqueeze(0).expand(B_ids, -1)                # (B_ids, Ls)

            # FULL model-assigned suffix distributions over vocab at each suffix position
            # pick the distribution at the timestep where each suffix token is predicted
            V = logits_suffix.size(-1)
            model_suffix_log_probs = log_probs_suffix.gather(
                dim=1,
                index=positions_logits.unsqueeze(-1).expand(-1, -1, V),
            )  # (B_ids, Ls, V)

            model_suffix_probs = model_suffix_log_probs.exp()        # (B_ids, Ls, V)
            # average across batch -> (Ls, V), one distribution per suffix position
            model_suffix_probs_mean = model_suffix_probs.mean(dim=0)  # (Ls, V)
            # stash on CPU for later feature extraction
            model_suffix_probs_cpu = model_suffix_probs_mean.detach().cpu()

            # log p(s_j | prompt + s_<j) for each batch, each suffix position
            suffix_token_logprobs = log_probs_suffix[
                torch.arange(B_ids, device=dev).unsqueeze(1),
                positions_logits,
                token_ids_expand,
            ]  # (B_ids, Ls)


            # schedule weights along suffix positions
            nt_sched = min(Ls, nt)          # "nt": first nt suffix positions
            weights = torch.ones(Ls, device=dev)  # default weight = 1.0
            if nt_sched > 0:
                # lower values at the start, gradually increasing up to 1.0
                weight_schedule = torch.linspace(0.3, 1.0, steps=nt_sched, device=dev)
                weights[:nt_sched] = weight_schedule

            # apply weights and average over batch
            weighted_logprobs = (suffix_token_logprobs * weights.unsqueeze(0)).sum(dim=-1)  # (B_ids,)
            suffix_joint_logprob = weighted_logprobs.mean()                                 # scalar

            # negative-signed multiplier: encourages higher joint suffix log-prob
            suffix_logprob_weight = 0.01
            suffix_logprob_term = -suffix_logprob_weight * suffix_joint_logprob

    # include the suffix joint log-prob term in the loss
    total_loss = base_loss + suffix_logprob_term
    
    return total_loss

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset

def load_prompts_unpadded(tokenizer, args):
    """
    Returns DataLoader where each batch is:
      {
        "input_ids": list of 1D LongTensors (prompts, no padding),
        "prompt_lens": LongTensor (B,)
      }
    """
    ds = load_dataset("tatsu-lab/alpaca", split="train", cache_dir=args["data_dir"])

    # Subsample for speed
    if "sample_size" in args and args["sample_size"] is not None and args["sample_size"] < len(ds):
        ds = ds.shuffle(seed=42).select(range(args["sample_size"]))

    def collate(batch):
        texts = [ex["instruction"] for ex in batch]
        enc = tokenizer(
            texts,
            padding=False,
            truncation=True,
            max_length=args["max_length"],
        )
        prompts = [torch.tensor(ids, dtype=torch.long) for ids in enc["input_ids"]]
        prompt_lens = [len(p) for p in prompts]

        return {
            "input_ids": prompts,  # list of (Li,)
            "prompt_lens": torch.tensor(prompt_lens, dtype=torch.long),
        }

    num_workers = max(2, os.cpu_count() // 2)
    return DataLoader(
        ds,
        batch_size=args["batch_size"],
        shuffle=True,
        pin_memory=True,
        num_workers=num_workers,
        persistent_workers=True,
        collate_fn=collate,
    )

dataloader_args = {
    "data_dir": "/kaggle/working/datasets",
    "batch_size": 1,
    "max_length": 128,
    "sample_size": 128,
}

dataloader = load_prompts_unpadded(tokenizer, dataloader_args)


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
def generate_with_suffix(
    model,
    tokenizer,
    prompt_ids: torch.Tensor,        # (L_prompt,)
    suffix_token_ids: torch.Tensor,  # (Ls,)
    max_new_tokens: int = 50,
):
    """
    Concatenate [prompt][suffix] and let the model generate continuation.
    """
    device = next(model.parameters()).device

    full_input_ids = torch.cat(
        [prompt_ids, suffix_token_ids.to(prompt_ids.device)],
        dim=0
    ).unsqueeze(0).to(device)  # (1, L_total)

    attn_mask = torch.ones_like(full_input_ids, dtype=torch.long, device=device)

    model.eval()
    with torch.no_grad():
        generated = model.generate(
            input_ids=full_input_ids,
            attention_mask=attn_mask,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

    return tokenizer.decode(generated[0], skip_special_tokens=True)

In [ ]:
import torch.nn.functional as F

def log_suffix_token_probs(
    model,
    tokenizer,
    prompt_ids: torch.Tensor,        # (L_prompt,)
    suffix_token_ids: torch.Tensor,  # (Ls,)
):
    """
    For each position k in the suffix, compute P(suffix[k] | prompt + suffix[:k])
    and print it.
    """
    device = next(model.parameters()).device
    model.eval()

    prompt_ids = prompt_ids.to(device)
    suffix_token_ids = suffix_token_ids.to(device)

    with torch.no_grad():
        for k in range(suffix_token_ids.size(0)):
            ctx_suffix = suffix_token_ids[:k]      # (k,)
            ctx = torch.cat([prompt_ids, ctx_suffix], dim=0)  # (L_prompt + k,)

            inp = ctx.unsqueeze(0)  # (1, L_ctx)
            msk = torch.ones_like(inp, dtype=torch.long, device=device)

            out = model(
                input_ids=inp,
                attention_mask=msk,
                use_cache=False,
                return_dict=True,
            )
            logits_next = out.logits[:, -1, :]  # (1, V)
            probs_next = F.softmax(logits_next, dim=-1)  # (1, V)

            tok_id = suffix_token_ids[k]
            prob = probs_next[0, tok_id].item()
            tok_str = tokenizer.decode([tok_id])

            print(f"  pos {k:2d}, token {tok_id:5d} ({repr(tok_str)}), prob: {prob:.6f}")

In [ ]:
from itertools import islice

device = next(model.parameters()).device
emb_layer = model.get_input_embeddings()

# Get one batch iterator for prompts
data_iter = iter(dataloader)
batch = next(data_iter)
prompts = batch["input_ids"]  # list of 1D tensors
prompt_ids = prompts[0]       # pick first prompt
print("Using this prompt for qualitative checks:")
print("  ", tokenizer.decode(prompt_ids, skip_special_tokens=True))
print()

with torch.no_grad():
    E_cpu = model.get_input_embeddings().weight.detach().cpu().float()  # (V, E)
    E_norm_cpu = F.normalize(E_cpu, dim=-1)  # (V, E), fp32 on CPU

for i in range(4):      # rounds
    for j in range(10): # epochs/checkpoints per round
        path = f"/kaggle/working/hotflip/rounds/trojai-0/suffix_saves/suffix_r{i}_e{j}.pt"

        try:
            suffix_z = read_suffix_pt(path)  # (Ls, E) on CPU
        except FileNotFoundError:
            print(f"[WARN] Missing file: {path}")
            continue

        print(f"\n=== round {i}, epoch {j} ===")
        print(f"Read {path} successfully. shape={tuple(suffix_z.shape)}")

        # Project to tokens + diagnostics
        suffix_token_ids = project_suffix_to_tokens_and_diagnostics(
            suffix_z,
            emb_layer,
            tokenizer,
            model,
            prompt_ids,
        )["suffix_token_ids"]  # LongTensor (Ls,)

        # Compute loss BEFORE projection (continuous suffix)
        batch_eos = {"input_ids": [torch.tensor([tokenizer.eos_token_id], dtype=torch.long)]}
        loss_before = compute_loss_for_suffix(
            model,
            emb_layer,
            batch_eos,
            suffix_z.to(device),
            n_tokens=10,
            amp_dtype=torch.float16,
            cos_reg_weight=1.0,
            E_norm_cpu=E_norm_cpu,
        )
        print(f"suffix loss (before projection): {loss_before.item():.6f}")

        # Build suffix_z AFTER projection: embeddings of discrete tokens
        suffix_z_proj = emb_layer(suffix_token_ids.to(device))  # (Ls, E)

        loss_after = compute_loss_for_suffix(
            model,
            emb_layer,
            batch_eos,
            suffix_z_proj,
            n_tokens=10,
            amp_dtype=torch.float16,
            cos_reg_weight=1.0,
            E_norm_cpu=E_norm_cpu,
        )
        print(f"suffix loss (after  projection): {loss_after.item():.6f}")

        # ---- Qualitative check: generate from a real prompt + suffix ----
        print("\nGenerated text with suffix (projected):")
        gen_text = generate_with_suffix(
            model,
            tokenizer,
            prompt_ids,
            suffix_token_ids,
            max_new_tokens=64,
        )
        print(gen_text)
        print()

        # ---- Per-suffix-token probabilities given the prompt ----
        print("Per-suffix-token next-token probabilities:")
        log_suffix_token_probs(
            model,
            tokenizer,
            prompt_ids,
            suffix_token_ids,
        )

        print("####################")


Using this prompt for qualitative checks:
   List three sites of historical importance in London.


=== round 0, epoch 0 ===
Read /kaggle/working/hotflip/rounds/suffix_r0_e0.pt successfully. shape=(10, 4096)
L2 distance between optimized embeddings and nearest token embeddings:
  min:  3.787844
  max:  4.888620
  mean: 4.140403
Cosine similarity of optimized embeddings to nearest tokens:
  min:  0.130694
  max:  0.297501
  mean: 0.202913

Projected discrete suffix token IDs: [20609, 1642, 7713, 10056, 10805, 29169, 3798, 1858, 7236, 25775]
Projected discrete suffix tokens: ['▁Portály', '".', 'Pol', '▁Consider', '▁causing', '²).', 'oreferrer', '▁Pl', '▁Fort', '▁Ari']
Projected suffix as text: 'Portály".Pol Consider causing²).oreferrer Pl Fort Ari'
suffix loss (before projection): 4.947422


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


suffix loss (after  projection): 5.449004

Generated text with suffix (projected):
List three sites of historical importance in London. Portály".Pol Consider causing²).oreferrer Pl Fort Ari. 1. The Tower of London 2. The Houses of Parliament 3. Buckingham Palace The Tower of London is a castle built by William the Conqueror in 1066. It was built to protect London from invaders. It was also used to keep prisoners. The Houses

Per-suffix-token next-token probabilities:
  pos  0, token 20609 ('Portály'), prob: 0.000000
  pos  1, token  1642 ('".'), prob: 0.000004
  pos  2, token  7713 ('Pol'), prob: 0.000015
  pos  3, token 10056 ('Consider'), prob: 0.000000
  pos  4, token 10805 ('causing'), prob: 0.000001
  pos  5, token 29169 ('²).'), prob: 0.000000
  pos  6, token  3798 ('oreferrer'), prob: 0.000000
  pos  7, token  1858 ('Pl'), prob: 0.000015
  pos  8, token  7236 ('Fort'), prob: 0.000002
  pos  9, token 25775 ('Ari'), prob: 0.000004
####################

=== round 0, epoch 1 ===
Rea

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


suffix loss (after  projection): 5.504870

Generated text with suffix (projected):
List three sites of historical importance in London. Portály".Mark Consider causing²).oreferrerMag Fort Ariane. 1. The Tower of London. 2. The Houses of Parliament. 3. The British Museum. 4. The British Library. 5. The National Gallery. 6. The National Portrait Gallery. 7. The Royal Opera House. 8. The Royal Academy of

Per-suffix-token next-token probabilities:
  pos  0, token 20609 ('Portály'), prob: 0.000000
  pos  1, token  1642 ('".'), prob: 0.000004
  pos  2, token  9802 ('Mark'), prob: 0.000019
  pos  3, token 10056 ('Consider'), prob: 0.000001
  pos  4, token 10805 ('causing'), prob: 0.000006
  pos  5, token 29169 ('²).'), prob: 0.000000
  pos  6, token  3798 ('oreferrer'), prob: 0.000000
  pos  7, token 19095 ('Mag'), prob: 0.000001
  pos  8, token  7236 ('Fort'), prob: 0.000017
  pos  9, token 25775 ('Ari'), prob: 0.000005
####################

=== round 0, epoch 2 ===
Read /kaggle/working/hotf

...

suffix loss (after  projection): 4.915522

Generated text with suffix (projected):
List three sites of historical importance in London. Sp More。 meter Screen P Mediabestanden M TMa.
































































Per-suffix-token next-token probabilities:
  pos  0, token  1706 ('Sp'), prob: 0.000304
  pos  1, token  5853 ('More'), prob: 0.000000
  pos  2, token 30267 ('。'), prob: 0.000000
  pos  3, token 11134 ('meter'), prob: 0.000000
  pos  4, token 22666 ('Screen'), prob: 0.000000
  pos  5, token   349 ('P'), prob: 0.000282
  pos  6, token 28574 ('Mediabestanden'), prob: 0.000000
  pos  7, token   341 ('M'), prob: 0.001537
  pos  8, token   323 ('T'), prob: 0.009132
  pos  9, token 21870 ('Ma'), prob: 0.000004
####################

=== round 3, epoch 8 ===
Read /kaggle/working/hotflip/rounds/suffix_r3_e8.pt successfully. shape=(10, 4096)
L2 distance between optimized embeddings and nearest token embeddings:
  min:  2.962456
  max:  7.777992
  mean: 4

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


suffix loss (after  projection): 4.915522

Generated text with suffix (projected):
List three sites of historical importance in London. Sp More。 meter Screen P Mediabestanden M TMa.
































































Per-suffix-token next-token probabilities:
  pos  0, token  1706 ('Sp'), prob: 0.000304
  pos  1, token  5853 ('More'), prob: 0.000000
  pos  2, token 30267 ('。'), prob: 0.000000
  pos  3, token 11134 ('meter'), prob: 0.000000
  pos  4, token 22666 ('Screen'), prob: 0.000000
  pos  5, token   349 ('P'), prob: 0.000282
  pos  6, token 28574 ('Mediabestanden'), prob: 0.000000
  pos  7, token   341 ('M'), prob: 0.001537
  pos  8, token   323 ('T'), prob: 0.009132
  pos  9, token 21870 ('Ma'), prob: 0.000004
####################

=== round 3, epoch 9 ===
Read /kaggle/working/hotflip/rounds/suffix_r3_e9.pt successfully. shape=(10, 4096)
L2 distance between optimized embeddings and nearest token embeddings:
  min:  2.738678
  max:  7.797936
  mean: 4

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


suffix loss (after  projection): 4.915522

Generated text with suffix (projected):
List three sites of historical importance in London. Sp More。 meter Screen P Mediabestanden M TMa.
































































Per-suffix-token next-token probabilities:
  pos  0, token  1706 ('Sp'), prob: 0.000304
  pos  1, token  5853 ('More'), prob: 0.000000
  pos  2, token 30267 ('。'), prob: 0.000000
  pos  3, token 11134 ('meter'), prob: 0.000000
  pos  4, token 22666 ('Screen'), prob: 0.000000
  pos  5, token   349 ('P'), prob: 0.000282
  pos  6, token 28574 ('Mediabestanden'), prob: 0.000000
  pos  7, token   341 ('M'), prob: 0.001537
  pos  8, token   323 ('T'), prob: 0.009132
  pos  9, token 21870 ('Ma'), prob: 0.000004
####################


In [ ]:
!ls /kaggle/working/hotflip/rounds

suffix_r0_e0.pt  suffix_r1_e0.pt  suffix_r2_e0.pt  suffix_r3_e0.pt
suffix_r0_e1.pt  suffix_r1_e1.pt  suffix_r2_e1.pt  suffix_r3_e1.pt
suffix_r0_e2.pt  suffix_r1_e2.pt  suffix_r2_e2.pt  suffix_r3_e2.pt
suffix_r0_e3.pt  suffix_r1_e3.pt  suffix_r2_e3.pt  suffix_r3_e3.pt
suffix_r0_e4.pt  suffix_r1_e4.pt  suffix_r2_e4.pt  suffix_r3_e4.pt
suffix_r0_e5.pt  suffix_r1_e5.pt  suffix_r2_e5.pt  suffix_r3_e5.pt
suffix_r0_e6.pt  suffix_r1_e6.pt  suffix_r2_e6.pt  suffix_r3_e6.pt
suffix_r0_e7.pt  suffix_r1_e7.pt  suffix_r2_e7.pt  suffix_r3_e7.pt
suffix_r0_e8.pt  suffix_r1_e8.pt  suffix_r2_e8.pt  suffix_r3_e8.pt
suffix_r0_e9.pt  suffix_r1_e9.pt  suffix_r2_e9.pt  suffix_r3_e9.pt


In [ ]:
refresh_repo()

/kaggle/working
Cloning into 'hotflip'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 341 (delta 11), reused 127 (delta 10), pack-reused 212 (from 1)
Receiving objects: 100% (341/341), 28.44 MiB | 21.22 MiB/s, done.
Resolving deltas: 100% (105/105), done.
/kaggle/working/hotflip
From https://github.com/jefri021/hotflip
 * branch            main       -> FETCH_HEAD
Already up to date.
